In [1]:
import sys, os
import folium
sys.path.append('..')
import flickrapi
import nationalparks.secrets as secrets
import nationalparks.parks as parks
import scrapper.fetch_images as fetch_images

In [2]:
from datetime import datetime

In [3]:
#filenames = os.listdir('../scrapper/data/geojson')
#filenames.sort()
#bboxes = {}
#for filename in filenames:
#    park_unit = filename.split('.')[0]
#    my_park = parks.Park(park_unit)
#    bboxes[park_unit] = my_park.get_bbox_string()

#image_finder = fetch_images.FlickrImageFinder(bboxes)
#image_finder.get_images_ids(erase=False)

In [4]:
!ls ../scrapper/data/image_infos/

acad_infos.csv


In [5]:
import pandas as pd
df = pd.read_csv('../scrapper/data/image_infos/' + 'acad' + '_infos.csv', index_col=0)
my_park = parks.Park('acad')

In [6]:
flickr = flickrapi.FlickrAPI(secrets.api_key, secrets.api_secret, format='parsed-json')

In [7]:
#owner = infos['photo']['owner']['username']
#dateposted = infos['photo']['dates']['taken']
#tags = [x['raw'] for x in infos['photo']['tags']['tag']]
#longitude = float(infos['photo']['location']['longitude'])
#latitude = float(infos['photo']['location']['latitude'])
#accuracy = infos['photo']['location']['accuracy']
#context = infos['photo']['location']['context']
#county = infos['photo']['location']['county']['_content']
#region = infos['photo']['location']['region']['_content']
#country = infos['photo']['location']['country']['_content']
#neighbourhood = infos['photo']['location']['neighbourhood']['_content']
#url = infos['photo']['urls']['url'][0]['_content']

In [8]:
def get_image_info(image_id):
    infos = flickr.photos.getInfo(photo_id=image_id)
    
    owner = infos['photo']['owner']['username']
    dateposted = infos['photo']['dates']['taken']
    tags = [x['raw'] for x in infos['photo']['tags']['tag']]
    longitude = float(infos['photo']['location']['longitude'])
    latitude = float(infos['photo']['location']['latitude'])
    #accuracy = infos['photo']['location']['accuracy']
    #context = infos['photo']['location']['context']
    #county = infos['photo']['location']['county']['_content']
    #region = infos['photo']['location']['region']['_content']
    #country = infos['photo']['location']['country']['_content']
    #neighbourhood = infos['photo']['location']['neighbourhood']['_content']
    url = infos['photo']['urls']['url'][0]['_content']
    
    return pd.Series({
        'owner':owner,
        'dateposted': dateposted,
        'tags': tags,
        'longitude': longitude,
        'latitude': latitude,
        #'accuracy': accuracy,
        #'context': context,
        #'county': county,
        #'region': region,
        #'country': country,
        #'neighbourhood': neighbourhood,
        'url': url})

#df[['owner','dateposted','tags','longitude','latitude','url']] = df['id'].apply(lambda x: get_image_info(x))

In [9]:
df['in_park'] = df.apply(lambda x: my_park.in_park(x), axis=1)

In [10]:
#df = df[df['in_park']]
#df.shape

In [11]:
import matplotlib.pyplot as plt

In [12]:
m = my_park.show_park()


for index, row in df[['latitude', 'longitude', 'in_park']].iterrows():
    
    if row['in_park']:
        folium.Circle(
            radius=10,
            location=[row['latitude'], row['longitude']],
            color='green',
            fill=False,
        ).add_to(m)
    else:
        folium.Circle(
            radius=10,
            location=[row['latitude'], row['longitude']],
            color='crimson',
            fill=False,
        ).add_to(m)
    
m